In [1]:
import csv
from sklearn.model_selection import train_test_split

import torch

# verify if GPU is available
if torch.cuda.is_available(): 
    
    device = torch.device("cuda")    
    print('There are %d GPU(s) available.' % torch.cuda.device_count())    
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [2]:
# initialize variables
import pandas as pd
import mysql.connector
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# mysql credentials
PASSWORD = os.getenv("PASSWORD")
USER = os.getenv("USER")


load_dotenv(override=True)


True

In [3]:
# connect to db
import mysql.connector

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user=USER,
  password=PASSWORD,
  database="mpp21"
)

mycursor = mydb.cursor()

In [4]:
categories = ["collection", "processing", "dissemination", "risk", "protection"]

article_ids = {}

for c in categories:
    article_ids[c] = {}

for c in categories:
    with open('./final_annotations/'+ c +'_org.csv', 'r') as f:
            reader = csv.reader(f)
            for r in reader:
                article_ids[c][r[0]] = r[1]

In [5]:
for i in article_ids.keys():
    print(i, len(article_ids[i]))

dissemination 229
protection 248
collection 250
processing 238
risk 200


In [209]:
# # get intersection
# intersection_over_all = set(article_ids["collection"].keys()).intersection(set(article_ids["processing"].keys()))
# intersection_over_all = intersection_over_all.intersection(set(article_ids["dissemination"].keys()))
# intersection_over_all = intersection_over_all.intersection(set(article_ids["risk"].keys()))
# intersection_over_all = intersection_over_all.intersection(set(article_ids["protection"].keys()))

In [210]:
# # also get union for all contents
# union_over_all = set(article_ids["collection"].keys()).union(set(article_ids["processing"].keys()))
# union_over_all = union_over_all.union(set(article_ids["dissemination"].keys()))
# union_over_all = union_over_all.union(set(article_ids["risk"].keys()))
# union_over_all = union_over_all.union(set(article_ids["protection"].keys()))

In [211]:
#len(intersection_over_all)

In [212]:
# # get the same test set, 20% of the smallest len, here 115
# len_test_set = round(0.2 * len(intersection_over_all))
# len_validation_set = round(0.1 * (len(intersection_over_all) - len_test_set))


In [213]:
# len_test_set

In [6]:
# TEST IF ONLY ABSTRACT
# db methods
def select_article(curs, db, aid):
    
    curs.execute("""SELECT title, content 
                FROM articles 
                WHERE article_id = '""" + aid + """' 
                """)
                #LIMIT 200""") 
    
    arts = {}
    result = curs.fetchall()
    for r in result:
        arts[aid] = {
            "title": r[0],
            "content": r[1],
            "verdict": article_ids[CLASSIFICATION][aid]
        }
    
    return arts[aid]



In [215]:
# import random 
# copy_intersection = list(intersection_over_all)
# test_set_ids, copy_intersection = get_set(len_test_set, copy_intersection)
# validation_set_ids, copy_intersection = get_set(len_validation_set, copy_intersection)

In [216]:
# import pandas as pd
# test_set_ids = list(pd.read_csv("final_annotations/test_set.csv")["header"])
# validation_set_ids = list(pd.read_csv("final_annotations/validation_set.csv")["header"])

In [217]:
# validation_set_ids

In [218]:
# """
#     TO DO:
#     save test and validation ids when running full thing

# """

# ONE ROUND OF BINARY CLASSIFICATION

In [129]:
CLASSIFICATION = "risk"

In [14]:
contents = {}

for a in article_ids[CLASSIFICATION]:#.keys():
    temp = select_article(mycursor, mydb, a)
    
    contents[a] = {
        "id": a,
        "label": int(temp["verdict"]),
        "text": temp["title"] + " " + temp["content"]
    }

    
# split into three sets

training_data = {}
validation_data = {}
test_data = {}

for a in contents.keys():
    if a in test_set_ids:
        test_data[a] = contents[a]
    elif a in validation_set_ids:
        validation_data[a] = contents[a]
    else:
        training_data[a] = contents[a]

NameError: name 'test_set_ids' is not defined

In [130]:
len(article_ids[CLASSIFICATION])

200

In [131]:
def get_set(cnt, ids):
    
    copy = ids
    
    random_ids = []
    
#     if len(keys) == 0:
#         return []
    
    for i in range(cnt):
        choice = random.choice(copy)
        random_ids.append(choice)
        copy.remove(choice)
        
    return sorted(random_ids), copy

In [132]:
import random

In [133]:
# len_test = round(0.1 * len(article_ids[CLASSIFICATION]))
# len_valid = round(0.1 * len(article_ids[CLASSIFICATION]))
# len_train = len(article_ids[CLASSIFICATION]) - len_test - len_valid

ones = [i for i in article_ids[CLASSIFICATION].keys() if int(article_ids[CLASSIFICATION][i]) == 1]
zeroes = [i for i in article_ids[CLASSIFICATION].keys() if int(article_ids[CLASSIFICATION][i]) == 0]
print(len(zeroes))
LEN_ONES = len(ones)
LEN_ZEROES = len(zeroes)

test_data, ones = get_set(round(0.1 * LEN_ONES), ones)# get 1s
print(len(test_data))
temp, zeroes = get_set(round(0.1 * LEN_ZEROES), zeroes) # get 0s
for i in temp:
    test_data.append(i)
    
validation_data, ones = get_set(round(0.1 * LEN_ONES), ones)# get 1s
print(len(validation_data))
temp, zeroes = get_set(round(0.1 * LEN_ZEROES), zeroes) # get 0s
for i in temp:
    validation_data.append(i)
    
training_data = ones
for i in zeroes:
    training_data.append(i)

155
4
4


In [134]:
final_test_data = {}
final_training_data = {}
final_validation_data = {}

for i in test_data:
    temp = select_article(mycursor, mydb, i)
    final_test_data[i] = {
        "id": i,
        "label": int(temp["verdict"]),
        "text": temp["title"] + " " + temp["content"]
    }
    
for i in training_data:
    temp = select_article(mycursor, mydb, i)
    final_training_data[i] = {
        "id": i,
        "label": int(temp["verdict"]),
        "text": temp["title"] + " " + temp["content"]
    }
    
for i in validation_data:
    temp = select_article(mycursor, mydb, i)
    final_validation_data[i] = {
        "id": i,
        "label": int(temp["verdict"]),
        "text": temp["title"] + " " + temp["content"]
    }

In [135]:
df_test = pd.DataFrame.from_dict(final_test_data).T
df_training = pd.DataFrame.from_dict(final_training_data).T
df_validation = pd.DataFrame.from_dict(final_validation_data).T

print('Number of test sentences: {:,}'.format(df_test.shape[0]))
print('Number of validation sentences: {:,}'.format(df_validation.shape[0]))
print('Number of training sentences: {:,}'.format(df_training.shape[0]))

df_training.sample(10)

Number of test sentences: 20
Number of validation sentences: 20
Number of training sentences: 160


,id,label,text
TDP_2459,TDP_2459,0,The thieves of our privacy My family had its s...
TDP_306,TDP_306,0,Otaki firm goes global to foil cyber-criminals...
TS_6428,TS_6428,0,Ensuring smart home features don't become a bu...
NYT_2467,NYT_2467,0,Don't limit speech in France FULL TEXTOn Tuesd...
SMH_6195,SMH_6195,1,AI won't rule humans if we take charge What do...
DT_6069,DT_6069,0,The new digital etiquetttte for t grown-ups Av...
TDP_1808,TDP_1808,0,Are we all facing an uncertain future? A whiff...
NYT_1548,NYT_1548,1,GPS and the law FULL TEXTLast week brought fre...
TDP_1667,TDP_1667,0,Hospitals' photo ban a delusion Hospitals must...
TS_7063,TS_7063,0,Which Ontario workers are most at risk for COV...


In [136]:
# df_test["id"].to_csv('final_annotations/test_set.csv', index=False)
# df_validation["id"].to_csv('final_annotations/validation_set.csv', index=False)   

In [137]:
from transformers import BertTokenizer

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [138]:
from keras.preprocessing.sequence import pad_sequences

In [139]:
def prepare_data(data): # a pandas frame
    
    sentences = data.text.values
    labels = data.label.values
    
    input_ids = []

    for sent in sentences:
        encoded_sent = tokenizer.encode(
                            sent,                      
                            add_special_tokens = True, 
                            max_length = 512,
                       )


        input_ids.append(encoded_sent)

    MAX_LEN = 512

    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                              value=0, truncating="post", padding="post")
    
    attention_masks = []

    for sent in input_ids:

        att_mask = [int(token_id > 0) for token_id in sent]

        attention_masks.append(att_mask)
    
    return input_ids, labels, attention_masks

In [140]:
# # split into train and test sets (0.8 training, 0.2 test)
# def split_data(inputs, labels, masks, split):

#     train_data, test_data, train_labels, test_labels = train_test_split(inputs, labels, 
#                                                                 random_state=2018, test_size=split)
#     # split masks
#     train_masks, test_masks, _, _ = train_test_split(masks, labels,
#                                                  random_state=2018, test_size=split)
    
#     return train_data, test_data, train_labels, test_labels, train_masks, test_masks

In [141]:
#input_ids, labels, attention_masks = prepare_data(df)

In [142]:
train_inputs, train_labels, train_masks = prepare_data(df_training)
test_inputs, test_labels, test_masks = prepare_data(df_test)
validation_inputs, validation_labels, validation_masks = prepare_data(df_validation)

In [143]:
# cast labels # if still needed
import numpy as np

x, y, z = [], [], []

[x.append(i) for i in train_labels]
[y.append(i) for i in validation_labels]
[z.append(i) for i in test_labels]

train_labels = np.array(x)
validation_labels = np.array(y)
test_labels = np.array(z)

In [144]:
# convert inputs and labels into torch tensors
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

test_inputs = torch.tensor(test_inputs)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_masks)

In [145]:
print("train\t\t", len(train_inputs))
print("validation\t", len(validation_inputs))
print("test\t\t", len(test_inputs))

train		 160
validation	 20
test		 20


In [146]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16 # 32 #16 # 32

# DataLoader for training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# DataLoader for validation set
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# DataLoader for the test set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [147]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# load BertForSequenceClassification (pre-trained BERT model with a single linear classif. layer)

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # 12-layer BERT model, uncased vocab (TODO: uncase articles)
    # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # number of output labels
                        # 2: binary classification
                        # >2: multi-class   
    output_attentions = False, # return attention weights
    output_hidden_states = False, # return hidden states
)

# run on GPU (uncomment if GPU available on server)
# model.cuda()

In [148]:
# get model's parameters
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
    
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
    
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [149]:
# AdamW 
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5,
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup

# training epochs: recommended 2-4
epochs = 15#10#0 ## CHANGE TO 4 - 8 - 10

# training steps = batches * epochs
total_steps = len(train_dataloader) * epochs

# create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            # default value in run_glue.py
                                            num_training_steps = total_steps)

In [150]:
# compute accuracy of predictions
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [151]:
import time
import datetime
import pickle

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
   
    elapsed_rounded = int(round((elapsed)))

    return str(datetime.timedelta(seconds=elapsed_rounded))

In [152]:
CLASSIFICATION

'risk'

In [153]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# store average loss after each epoch
loss_values = []


models = {}
stats = {}


for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # full pass over the training set    
    print()
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')    
    t0 = time.time() # measure epoch length
    total_loss = 0 # reset total loss for current epoch
    
    # place model in training mode
    model.train()    

    # for each batch
    for step, batch in enumerate(train_dataloader):        
        #print("Step: ", step)
        # updated on progress every 40 batches
        if step % 10 == 0 and not step == 0:

            elapsed = format_time(time.time() - t0) # compute elapsed time
            
            # report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))        
            
        # unpack training batch from DataLoader
        # copy each tensor to GPU ('to' method)
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)        
        
        # clear previously calculated gradients before a backward pass
        model.zero_grad()                
        
        # forward pass (i.e., evaluate the model on this training batch)
        # this returns the loss (and not the model output), because we provided the `labels`

        # documentation for model function:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # get loss value
        loss = outputs[0]        
        
        # accumulate training loss over all batches (to compute average loss at the end)
        total_loss += loss.item()     
        
        # backward pass to calculate gradients
        loss.backward()        
        
        # clip the norm of the gradients to 1.0 to prevent the 'exploding gradients' problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)        
        
        # update parameters (the optimizer dictates the update rule based on gradients, learning rate, etc.)
        # take a step using the computed gradient
        optimizer.step() 
        
        # update the learning rate.
        scheduler.step()    
    
    # calculate average loss over training data
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # store the loss value to plot the learning curve
    loss_values.append(avg_train_loss)    
    
    print()
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    

    # after each training epoch, measure performance on validation set
    print()
    print("Running Validation...")    
    t0 = time.time()   
    
    ### STORE MODEL
    models[epoch_i] = model
    fn = "models/"+CLASSIFICATION+"/model_r2_e" + str(epoch_i) + ".sav"
    pickle.dump(model, open(fn, 'wb'))
    
    # place model in evaluation mode (dropout layers behave differently than during training)
    model.eval()    
    
    # tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0    
    
    # evaluate data for one epoch
    for batch in validation_dataloader:
        
        # add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # unpack inputs from DataLoader
        b_input_ids, b_input_mask, b_labels = batch
        
        
        # do not compute or store gradients to save memory and speedup validation
        with torch.no_grad():                    
            # forward pass, calculate logit predictions
                # will return logits (not the loss, because we have not provided labels)
            # token_type_ids = segment ids (differentiates between sentence 1 and 2 in 2-sentence tasks)
            
            
            # documentation for model
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # get logits (output) from model
        # values prior to applying an activation function (e.g., softmax)
        logits = outputs[0]    
        
        # move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # compute accuracy for current batch 
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # accumulate accuracy
        eval_accuracy += tmp_eval_accuracy        
        # track batches
        nb_eval_steps += 1    
        # report final accuracy for current validation run
    print("  Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    
    stats[epoch_i] = {
        "acc": round(eval_accuracy/nb_eval_steps, 5),
        "avg-loss": round(avg_train_loss, 5)
    }
    print("")
print("Training complete!")


======== Epoch 1 / 15 ========
Training...

  Average training loss: 0.55
  Training epoch took: 0:03:08

Running Validation...
  Accuracy: 0.78125
  Validation took: 0:00:06


======== Epoch 2 / 15 ========
Training...

  Average training loss: 0.52
  Training epoch took: 0:03:07

Running Validation...
  Accuracy: 0.78125
  Validation took: 0:00:07


======== Epoch 3 / 15 ========
Training...

  Average training loss: 0.45
  Training epoch took: 0:03:08

Running Validation...
  Accuracy: 0.75000
  Validation took: 0:00:06


======== Epoch 4 / 15 ========
Training...

  Average training loss: 0.35
  Training epoch took: 0:03:05

Running Validation...
  Accuracy: 0.46875
  Validation took: 0:00:06


======== Epoch 5 / 15 ========
Training...

  Average training loss: 0.24
  Training epoch took: 0:03:05

Running Validation...
  Accuracy: 0.50000
  Validation took: 0:00:06


======== Epoch 6 / 15 ========
Training...

  Average training loss: 0.16
  Training epoch took: 0:03:03

Running 

In [154]:
df_stats = pd.DataFrame.from_dict(stats)
df_stats.T

,acc,avg-loss
0,0.78125,0.55114
1,0.78125,0.52356
2,0.75000,0.44836
3,0.46875,0.34731
4,0.50000,0.24083
5,0.56250,0.16246
6,0.59375,0.11350
7,0.40625,0.06308
8,0.40625,0.04362
9,0.40625,0.03264


In [166]:
import pickle
def load_model(m):
    return pickle.load(open("models/"+CLASSIFICATION+"/model_r2_e" + str(m) + ".sav", 'rb'))

In [206]:
model = load_model(14) # CHANGE NUMBER (uses index)

In [207]:
# prediction on test set
print('Predicting labels for {:,} test documents...'.format(len(test_inputs)))

# place model in evaluation mode
model.eval()

# tracking variables 
predictions, true_labels = [], []

# predict 
for batch in test_dataloader:
    # add batch to GPU (if available)
    batch = tuple(t.to(device) for t in batch)
  
    # unpack inputs from DataLoader
    b_input_ids, b_input_mask, b_labels = batch
  
    with torch.no_grad():
        # forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask) 
        
    logits = outputs[0]  
    
    # move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)
    
print('\nDONE.')

Predicting labels for 20 test documents...

DONE.


In [208]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# for each batch
for i in range(len(true_labels)):
    # predictions are a 2-column ndarray (one for 0, one for 1)
    # pick label with highest value and turn it into 1 or 0

    pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
    # calculate and store coef for current batch
    matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
    matthews_set.append(matthews)

Calculating Matthews Corr. Coef. for each batch...


In [209]:
# combine the predictions for each batch into a single list of 0s and 1s

flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# combine the correct labels for each batch into a single list
flat_true_labels = [item for sublist in true_labels for item in sublist]

# calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('MCC: %.3f' % mcc)

MCC: -0.367


In [210]:
from sklearn import metrics

In [211]:
x_data = flat_true_labels
y_data = flat_predictions

tn, fp, fn, tp = get_conf_matrix(x_data, y_data)#confusion_matrix(np.array(flat_true_labels), flat_predictions).ravel()
print(
    "\nTN:", tn,
    "\nTP:", tp,
    "\nFN:", fn,
    "\nFP:", fp    
)

recall = round(get_recall(x_data, y_data), 3)
precision = round(get_precision(x_data, y_data), 3)
f1 = round(get_f1_score(x_data, y_data), 3)

print("Recall:", recall)
print("Precision:", precision)
print("F1-score:", f1)
fpr, tpr, thresholds = metrics.roc_curve(x_data, y_data, pos_label=1)
print(metrics.auc(fpr, tpr))
print("FPR", fpr)
print("TPR", tpr)


TN: 9 
TP: 0 
FN: 4 
FP: 7
Recall: 0.281
Precision: 0.346
F1-score: 0.31
0.28125
FPR [0.     0.4375 1.    ]
TPR [0. 0. 1.]


In [41]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

def get_conf_matrix(x, y):
    return confusion_matrix(np.array(x), y).ravel()

def get_recall(x, y):
    return recall_score(x, y, average='macro')
    
def get_precision(x, y):
    return precision_score(x, y, average='macro')

def get_f1_score(x, y):
    return f1_score(x, y, average='macro')

# TODO: predict function for the 26k articles

In [43]:
def select_articles_for_prediction(curs, db, n):
    
    curs.execute("""SELECT article_id, title, content 
                FROM articles 
                WHERE newspaper = '""" + n + """' 
                AND is_privacy = 'privacy'
                """)
                #LIMIT 200""") AND year(DATE) != '2010'
    
    arts = {}
    result = curs.fetchall()
    for r in result:
        arts[r[0]] = {
            "title": r[1],
            "content": r[2]
        }
    
    return arts



In [44]:
def parse_content(d):
    # tokenize
    encoded_sent = tokenizer.encode(d, add_special_tokens = True, max_length = 512)
    
    # pad
    input_id = pad_sequences([encoded_sent], maxlen=512, 
                          dtype="long", truncating="post", padding="post")
    #print(input_id)
    
    # create attention mask
    seq_mask = [float(i>0) for i in input_id[0]]

    # convert to tensors
    prediction_input = torch.tensor(input_id)
    prediction_mask = torch.tensor([seq_mask])
    
    res = {
        "input": prediction_input,
        "mask": prediction_mask
    }
    return res


# function to get category label
def is_category(res):
    
#     # tokenize
#     encoded_sent = tokenizer.encode(d, add_special_tokens = True, max_length = 512)
    
#     # pad
#     input_id = pad_sequences([encoded_sent], maxlen=512, 
#                           dtype="long", truncating="post", padding="post")
#     #print(input_id)
    
#     # create attention mask
#     seq_mask = [float(i>0) for i in input_id[0]]

#     # convert to tensors
#     prediction_input = torch.tensor(input_id)
#     prediction_mask = torch.tensor([seq_mask])
    output = MODEL(res["input"], token_type_ids=None, 
                      attention_mask=res["mask"]) 
        
    logits = output[0]  
    
    # move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    
    # store predictions and true labels
    verdict = np.argmax(logits[0])

    return int(verdict)


In [198]:
import time
from datetime import datetime


In [212]:
MODEL = model

In [286]:
n = "TG"
articles_to_predict = select_articles_for_prediction(mycursor, mydb, n)

In [287]:
len(articles_to_predict)

7435

In [288]:
contents_to_predict = {}

for a in articles_to_predict.keys():
    
    contents_to_predict[a] = {
        "id": a,
        "text": articles_to_predict[a]["title"] + " " + articles_to_predict[a]["content"]
    }

print(len(contents_to_predict))

7435


In [289]:
parsed_contents = {}

for i in contents_to_predict.keys():
    parsed_contents[i] = parse_content(contents_to_predict[i]["text"])
print(len(parsed_contents))

7435


In [290]:
verdicts = {}

for i in parsed_contents.keys():
    verdicts[i] = {
      #  "collection": -1,
      #  "processing": -1,
        CLASSIFICATION: -1
    }
print(len(verdicts))

7435


In [291]:
categ = 0
non_categ = 0
ctr = 0


now = datetime.now()

print(now.strftime('%H: %M: %S %p'))

for i in parsed_contents.keys():
    v = is_category(parsed_contents[i])
    verdicts[i][CLASSIFICATION] = v
    
    ctr += 1
    if ctr % 300 == 0:
        print("Predicted:", ctr, "Left:", len(parsed_contents) - ctr)
        now = datetime.now()

        print(now.strftime('%H: %M: %S %p'))
    if v == 1:
        categ += 1
    elif v == 0:
        non_categ += 1
        
now = datetime.now()

print(now.strftime('%H: %M: %S %p'))

20: 02: 45 PM
Predicted: 300 Left: 7135
20: 04: 15 PM
Predicted: 600 Left: 6835
20: 05: 45 PM
Predicted: 900 Left: 6535
20: 07: 15 PM
Predicted: 1200 Left: 6235
20: 08: 46 PM
Predicted: 1500 Left: 5935
20: 10: 14 PM
Predicted: 1800 Left: 5635
20: 11: 40 PM
Predicted: 2100 Left: 5335
20: 13: 14 PM
Predicted: 2400 Left: 5035
20: 14: 42 PM
Predicted: 2700 Left: 4735
20: 16: 12 PM
Predicted: 3000 Left: 4435
20: 17: 41 PM
Predicted: 3300 Left: 4135
20: 19: 10 PM
Predicted: 3600 Left: 3835
20: 20: 36 PM
Predicted: 3900 Left: 3535
20: 22: 06 PM
Predicted: 4200 Left: 3235
20: 23: 35 PM
Predicted: 4500 Left: 2935
20: 25: 04 PM
Predicted: 4800 Left: 2635
20: 26: 34 PM
Predicted: 5100 Left: 2335
20: 28: 02 PM
Predicted: 5400 Left: 2035
20: 29: 32 PM
Predicted: 5700 Left: 1735
20: 31: 01 PM
Predicted: 6000 Left: 1435
20: 32: 29 PM
Predicted: 6300 Left: 1135
20: 33: 58 PM
Predicted: 6600 Left: 835
20: 35: 27 PM
Predicted: 6900 Left: 535
20: 36: 57 PM
Predicted: 7200 Left: 235
20: 38: 26 PM
20: 39: 

In [292]:
# # dump verdicts to update all at once
import json
with open("verdicts/"+ CLASSIFICATION +"_"+ n +".json", "w") as outfile:  
    json.dump(verdicts, outfile) 

In [293]:
print("Categ:", categ)
print(round(categ * 100 / len(parsed_contents.keys()), 2))

print("Non-categ:", non_categ)
print(round(non_categ * 100 / len(parsed_contents.keys()), 2))

Categ: 1484
19.96
Non-categ: 5951
80.04


In [55]:
# SAVE MODEL USED
#pickle.dump(MODEL, open("models/final_"+ CLASSIFICATION +".sav", 'wb'))

In [ ]:
# for a in verdicts.keys():
#     update_categ(mycursor, mydb, json.dumps(verdicts[a]), a)
